<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# MAT281 - Laboratorio N°03





**Objetivo**: Aplicar técnicas avanzadas de manipulación y análisis de datos con pandas sobre un conjunto real de datos de contenido de Netflix, reforzando buenas prácticas y métodos eficientes sin recurrir a `groupby`, `merge`, `pivot`, ni `join`.



**Dataset**:

Trabajaremos con el archivo `netflix_titles.csv`, que contiene información sobre los títulos disponibles en la plataforma Netflix hasta el año 2021.

| Variable       | Clase     | Descripción                                                                 |
|----------------|-----------|------------------------------------------------------------------------------|
| show_id        | caracter  | Identificador único del título en el catálogo de Netflix.                   |
| type           | caracter  | Tipo de contenido: 'Movie' o 'TV Show'.                                     |
| title          | caracter  | Título del contenido.                                                       |
| director       | caracter  | Nombre del director (puede ser nulo).                                       |
| cast           | caracter  | Lista de actores principales (puede ser nulo).                              |
| country        | caracter  | País o países donde se produjo el contenido.                                |
| date_added     | fecha     | Fecha en la que el título fue agregado al catálogo de Netflix.              |
| release_year   | entero    | Año de lanzamiento original del título.                                     |
| rating         | caracter  | Clasificación por edad (por ejemplo: 'PG-13', 'TV-MA').                      |
| duration       | caracter  | Duración del contenido (minutos o número de temporadas para series).        |
| listed_in      | caracter  | Categorías o géneros en los que está clasificado el contenido.              |
| description    | caracter  | Breve sinopsis del contenido.                                               |




In [1]:
import pandas as pd

# Cargar datos
df = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...



### Parte 1: Limpieza y preparación

1. Revisar y describir el dataset:

   * ¿Cuántas filas y columnas tiene?
   * ¿Qué tipos de datos hay?
   * ¿Cuántos valores nulos hay por columna?

2. Transformar la columna `date_added` a tipo fecha.

3. Crear columnas auxiliares con `assign`:

   * Año (`year_added`)
   * Mes (`month_added`)



In [24]:
#FIXME
# PARTE 1
print("cantidad de filas: ", len(df), "; cantidad de columnas: ", len(df.columns))
print("cantidad de filas y columnas es:",df.shape)


cantidad de filas:  8807 ; cantidad de columnas:  12
cantidad de filas y columnas es: (8807, 12)


In [25]:
print("Los tipos de datos que hay son:\n",df.dtypes)




Los tipos de datos que hay son:
 show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


In [4]:
#valores nulos por columna

print("Valores nulos por columna:\n", df.isna().sum())

Valores nulos por columna:
 show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


In [5]:
# PARTE 2: Transformar la columna date_added a tipo fecha (robusto)
# Limpia espacios y vacíos si la columna es de texto
if df["date_added"].dtype == "O":
    df["date_added"] = df["date_added"].str.strip()
    # convierte cadenas vacías en NA antes de parsear
    df["date_added"] = df["date_added"].replace(r"^\s*$", pd.NA, regex=True)

# Parseo con coerce para no romper si hay valores inválidos
df["date_added"] = pd.to_datetime(df["date_added"], errors="coerce")

print(df["date_added"].head())  # vista rápida, evita imprimir toda la serie

0   2021-09-25
1   2021-09-24
2   2021-09-24
3   2021-09-24
4   2021-09-24
Name: date_added, dtype: datetime64[ns]


In [6]:
# PARTE 3: crear columnas año/mes con enteros "nullable"
df = df.assign(
    year_added = df["date_added"].dt.year.astype("Int64"),
    month_added = df["date_added"].dt.month.astype("Int64")
)

print(df[["date_added", "year_added", "month_added"]].head())

  date_added  year_added  month_added
0 2021-09-25        2021            9
1 2021-09-24        2021            9
2 2021-09-24        2021            9
3 2021-09-24        2021            9
4 2021-09-24        2021            9


# Parte 2: Técnicas avanzadas de pandas

4. Utilizar `.loc` para seleccionar películas (`type == 'Movie'`) que fueron agregadas después del año 2018.

5. Utilizar `str.contains()` y `str.extract()`:

   * Filtrar títulos que contienen la palabra 'love' (sin distinguir mayúsculas/minúsculas).
   * Extraer la duración en minutos para las películas desde la columna `duration`.

6. Aplicar `explode()` sobre la columna `listed_in` para obtener una fila por cada género.

7. Obtener un top 10 de géneros más frecuentes utilizando `value_counts()`.

8. Aplicar `where()` y `mask()` para marcar las películas de más de 120 minutos como contenido largo en una nueva columna.

9. Utilizar `.loc` para filtrar películas que cumplen con:

   * Más de 100 minutos de duración.
   * Rating igual a `'R'`.
   * País igual a `'United States'`.

10. Utilizar `.style` para formatear visualmente el top 10 de películas más largas.



### Pregunta Desafío

11. ¿Cuáles son las combinaciones más frecuentes de género y rating en el dataset?
    (Sugerencia: utilizar `value_counts` con `subset=["genre", "rating"]` después de aplicar `explode()`).



### Bonus: Análisis de duplicados y limpieza

12. ¿Existen películas con el mismo nombre (`title`) pero con distinto año de lanzamiento (`release_year`)?
13. ¿Cuántos títulos únicos hay en total en la columna `title`?





In [7]:
#FIXME
#Parte 4: Utilizar .loc para seleccionar películas (type == 'Movie') que fueron agregadas después del año 2018
condicion = (df["type"] == "Movie") & (df["year_added"] > 2018)
pelis = df.loc[condicion, ["type", "date_added", "year_added"]]
print(pelis.head())


     type date_added  year_added
0   Movie 2021-09-25        2021
6   Movie 2021-09-24        2021
7   Movie 2021-09-24        2021
9   Movie 2021-09-24        2021
12  Movie 2021-09-23        2021


In [8]:
#Parte 5: 
# Filtrar títulos que contienen 'love' (case-insensitive)
mask_love = df["title"].str.contains("love", case=False, na=False)
love_titles = df.loc[mask_love, ["title", "type", "duration"]]
print(love_titles.head())
print(len(love_titles))

# Extraer minutos SOLO para Movies (quedan <NA> para TV Shows)
mask_movie = df["type"].eq("Movie")
df.loc[mask_movie, "duration_min"] = (
    df.loc[mask_movie, "duration"].str.extract(r"(\d+)\s*min", expand=False).astype("Int64")
)

                         title     type   duration
25        Love on the Spectrum  TV Show  2 Seasons
158    Love Don't Cost a Thing    Movie    101 min
159             Love in a Puff    Movie    103 min
206  LSD: Love, Sex Aur Dhokha    Movie    112 min
227                Really Love    Movie     95 min
196


In [9]:
#Parte 6:
df_exploded = df.assign(genre = df["listed_in"].str.split(",")) \
                .explode("genre")
df_exploded["genre"] = df_exploded["genre"].str.strip()
df_exploded = df_exploded[df_exploded["genre"].notna() & (df_exploded["genre"] != "")]
print(df_exploded[["title", "genre"]].head())


                  title                   genre
0  Dick Johnson Is Dead           Documentaries
1         Blood & Water  International TV Shows
1         Blood & Water               TV Dramas
1         Blood & Water            TV Mysteries
2             Ganglands          Crime TV Shows


In [10]:
#Parte 7:
# Top 10 géneros más frecuentes
top10 = df_exploded["genre"].value_counts().head(10)
print(top10)


genre
International Movies        2752
Dramas                      2427
Comedies                    1674
International TV Shows      1351
Documentaries                869
Action & Adventure           859
TV Dramas                    763
Independent Movies           756
Children & Family Movies     641
Romantic Movies              616
Name: count, dtype: int64


In [11]:
#Parte 8:
# Asegurar duration_min (si no lo hiciste arriba)
mask_movie = df["type"].eq("Movie")
df.loc[mask_movie, "duration_min"] = (
    df.loc[mask_movie, "duration"].str.extract(r"(\d+)\s*min", expand=False).astype("Int64")
)

cond_long = mask_movie & df["duration_min"].gt(120)

# where: deja "Contenido largo" donde True y NaN donde False
df["contenido_largo_where"] = pd.Series("Contenido largo", index=df.index).where(cond_long)

# mask: parte de "No" y reemplaza a "Sí" donde True
df["contenido_largo_mask"] = "No"
df["contenido_largo_mask"] = df["contenido_largo_mask"].mask(cond_long, "Sí")

print(df.loc[mask_movie, ["title", "duration_min", "contenido_largo_where", "contenido_largo_mask"]].head())




                               title  duration_min contenido_largo_where  \
0               Dick Johnson Is Dead            90                   NaN   
6   My Little Pony: A New Generation            91                   NaN   
7                            Sankofa           125       Contenido largo   
9                       The Starling           104                   NaN   
12                      Je Suis Karl           127       Contenido largo   

   contenido_largo_mask  
0                    No  
6                    No  
7                    Sí  
9                    No  
12                   Sí  


In [12]:
#Parte 9:
# Normaliza rating por si trae espacios
df["rating_norm"] = df["rating"].astype(str).str.strip()

# Opción EXACTA (como escribiste):
cond = (
    df["type"].eq("Movie")
    & df["duration_min"].gt(100)
    & df["rating_norm"].eq("R")
    & df["country"].eq("United States")
)

# Opción INCLUYE EE.UU. (si tu dataset trae "United States, Canada"):
# cond = (
#     df["type"].eq("Movie")
#     & df["duration_min"].gt(100)
#     & df["rating_norm"].eq("R")
#     & df["country"].astype(str).str.contains(r"\bUnited States\b", na=False)
# )

pelis = df.loc[cond, ["title", "duration", "duration_min", "rating_norm", "country"]]
print(pelis.head())
print(len(pelis))




                           title duration  duration_min rating_norm  \
48                  Training Day  122 min           122           R   
81                          Kate  106 min           106           R   
131  Blade Runner: The Final Cut  117 min           117           R   
139           Do the Right Thing  120 min           120           R   
144                  House Party  104 min           104           R   

           country  
48   United States  
81   United States  
131  United States  
139  United States  
144  United States  
243


In [13]:
#Parte 10:

mask_movie = df["type"].eq("Movie")
# duration_min ya calculado arriba

top10_movies = (
    df.loc[mask_movie & df["duration_min"].notna(), ["title", "duration", "duration_min", "rating", "country"]]
      .sort_values("duration_min", ascending=False)
      .head(10)
      .reset_index(drop=True)
)
top10_movies["duration_min"] = top10_movies["duration_min"].astype(int)

# En notebook, haz que ESTA sea la última línea de la celda o usa display(top10_movies.style...)
top10_movies.style\
    .format({"duration_min": "{:,.0f} min"})\
    .bar(subset=["duration_min"], align="left")\
    .set_caption("Top 10 películas más largas")


,title,duration,duration_min,rating,country
0,Black Mirror: Bandersnatch,312 min,312 min,TV-MA,United States
1,Headspace: Unwind Your Mind,273 min,273 min,TV-G,nan
2,The School of Mischief,253 min,253 min,TV-14,Egypt
3,No Longer kids,237 min,237 min,TV-14,Egypt
4,Lock Your Girls In,233 min,233 min,TV-PG,nan
5,Raya and Sakina,230 min,230 min,TV-14,nan
6,Once Upon a Time in America,229 min,229 min,R,"Italy, United States"
7,Sangam,228 min,228 min,TV-14,India
8,Lagaan,224 min,224 min,PG,"India, United Kingdom"
9,Jodhaa Akbar,214 min,214 min,TV-14,India


In [14]:
#Parte 11:
top = (
    df_exploded[["genre", "rating"]]
      .value_counts(dropna=False)           # cuenta pares (genre, rating)
      .reset_index(name="count")            # a columnas
      .sort_values("count", ascending=False)
      .head(10)
)
print(top)


                    genre rating  count
0    International Movies  TV-MA   1130
1    International Movies  TV-14   1065
2                  Dramas  TV-MA    830
3  International TV Shows  TV-MA    714
4                  Dramas  TV-14    693
5  International TV Shows  TV-14    472
6                Comedies  TV-14    465
7               TV Dramas  TV-MA    434
8                Comedies  TV-MA    431
9                  Dramas      R    375


In [15]:
#Parte 12:

# ¿Títulos con distinto release_year?
g = df.groupby("title")["release_year"]
multi_mask = g.nunique() > 1
print("Títulos con >1 release_year (dataset completo):", int(multi_mask.sum()))

# Muestra rápida (primeros 20)
print(
    df.loc[df["title"].isin(multi_mask[multi_mask].index), ["title","release_year"]]
      .drop_duplicates()
      .sort_values(["title","release_year"])
      .head(20)
)

# ¿Cuántos títulos únicos hay en total?
print("Títulos únicos totales:", df["title"].nunique())


Títulos con >1 release_year (dataset completo): 0
Empty DataFrame
Columns: [title, release_year]
Index: []
Títulos únicos totales: 8807
